In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px
warnings.simplefilter('ignore')

from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords

%matplotlib inline

In [ ]:
!pip install openpyxl

In [ ]:
%pwd

'/content'

In [ ]:
df = pd.read_excel('/content/orders_data.xlsx')

In [ ]:
df.head(10)

,order_no,order_date,buyer,ship_city,ship_state,sku,description,quantity,item_total,shipping_fee,cod,order_status
0,405-9763961-5211537,"Sun, 18 Jul, 2021, 10:38 pm IST",Mr.,"CHANDIGARH,",CHANDIGARH,SKU: 2X-3C0F-KNJE,100% Leather Elephant Shaped Piggy Coin Bank |...,1,₹449.00,NaN,NaN,Delivered to buyer
1,404-3964908-7850720,"Tue, 19 Oct, 2021, 6:05 pm IST",Minam,"PASIGHAT,",ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,₹449.00,₹60.18,NaN,Delivered to buyer
2,171-8103182-4289117,"Sun, 28 Nov, 2021, 10:20 pm IST",yatipertin,"PASIGHAT,",ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,₹449.00,₹60.18,NaN,Delivered to buyer
3,405-3171677-9557154,"Wed, 28 Jul, 2021, 4:06 am IST",aciya,"DEVARAKONDA,",TELANGANA,SKU: AH-J3AO-R7DN,Pure 100% Leather Block Print Rectangular Jewe...,1,NaN,NaN,Cash On Delivery,Delivered to buyer
4,402-8910771-1215552,"Tue, 28 Sept, 2021, 2:50 pm IST",Susmita,"MUMBAI,",MAHARASHTRA,SKU: KL-7WAA-Z82I,Pure Leather Sling Bag with Multiple Pockets a...,1,"₹1,099.00",₹84.96,NaN,Delivered to buyer
5,406-9292208-6725123,"Thu, 17 Jun, 2021, 9:12 pm IST",Subinita,"HOWRAH,",WEST BENGAL,SKU: HH-FOWV-5YWO,Women's Trendy Pure Leather Clutch Purse | Lea...,1,₹200.00,NaN,NaN,Delivered to buyer
6,404-5794317-7737924,"Thu, 12 Aug, 2021, 8:03 pm IST",shailendra,"ORAI,",UTTAR PRADESH,SKU: TQ-OE6K-9DIK,Ultra Slim 100% Pure Leather Men's Wallet with...,1,NaN,NaN,Cash On Delivery,Returned to seller
7,405-8702211-4054722,"Wed, 29 Sept, 2021, 2:55 pm IST",Pratima,"BAREILLY,",UTTAR PRADESH,SKU: S1-A92Q-JU3X,100% Pure Leather Shantiniketan Clutch Purse: ...,1,₹399.00,₹84.96,Cash On Delivery,Delivered to buyer
8,171-1434812-8061163,"Sat, 13 Nov, 2021, 7:37 pm IST",Ipshita,"BENGALURU,",KARNATAKA,SKU: 3F-4R9N-Z8NJ,Set of 2 Pure Leather Block Print Round Jewelr...,1,₹399.00,₹84.96,NaN,Delivered to buyer
9,171-7954707-4463549,"Mon, 9 Aug, 2021, 4:47 pm IST",A.Jayaprada,"Bhilai,",CHHATTISGARH,SKU: NU-CKZ5-4O49,Pure Leather Sling Bag with Multiple Pockets a...,1,"₹1,099.00",NaN,NaN,Delivered to buyer


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   order_no      171 non-null    object
 1   order_date    171 non-null    object
 2   buyer         171 non-null    object
 3   ship_city     171 non-null    object
 4   ship_state    171 non-null    object
 5   sku           171 non-null    object
 6   description   171 non-null    object
 7   quantity      171 non-null    int64 
 8   item_total    153 non-null    object
 9   shipping_fee  145 non-null    object
 10  cod           47 non-null     object
 11  order_status  171 non-null    object
dtypes: int64(1), object(11)
memory usage: 16.2+ KB


In [ ]:
df.shape

(171, 12)

In [ ]:
def drop(df, *features):
    ''' To drop the features from a dataframe'''
    for i in features:
        df.drop(i, axis=1, inplace=True) #drop 함수 : 행 또는 열 제거 / axis=0 행, 1 열 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   order_no      171 non-null    object
 1   order_date    171 non-null    object
 2   buyer         171 non-null    object
 3   ship_city     171 non-null    object
 4   ship_state    171 non-null    object
 5   sku           171 non-null    object
 6   description   171 non-null    object
 7   quantity      171 non-null    int64 
 8   item_total    153 non-null    object
 9   shipping_fee  145 non-null    object
 10  cod           47 non-null     object
 11  order_status  171 non-null    object
dtypes: int64(1), object(11)
memory usage: 16.2+ KB


In [ ]:
df.head(10)

,order_no,order_date,buyer,ship_city,ship_state,sku,description,quantity,item_total,shipping_fee,cod,order_status
0,405-9763961-5211537,"Sun, 18 Jul, 2021, 10:38 pm IST",Mr.,"CHANDIGARH,",CHANDIGARH,SKU: 2X-3C0F-KNJE,100% Leather Elephant Shaped Piggy Coin Bank |...,1,₹449.00,NaN,NaN,Delivered to buyer
1,404-3964908-7850720,"Tue, 19 Oct, 2021, 6:05 pm IST",Minam,"PASIGHAT,",ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,₹449.00,₹60.18,NaN,Delivered to buyer
2,171-8103182-4289117,"Sun, 28 Nov, 2021, 10:20 pm IST",yatipertin,"PASIGHAT,",ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,₹449.00,₹60.18,NaN,Delivered to buyer
3,405-3171677-9557154,"Wed, 28 Jul, 2021, 4:06 am IST",aciya,"DEVARAKONDA,",TELANGANA,SKU: AH-J3AO-R7DN,Pure 100% Leather Block Print Rectangular Jewe...,1,NaN,NaN,Cash On Delivery,Delivered to buyer
4,402-8910771-1215552,"Tue, 28 Sept, 2021, 2:50 pm IST",Susmita,"MUMBAI,",MAHARASHTRA,SKU: KL-7WAA-Z82I,Pure Leather Sling Bag with Multiple Pockets a...,1,"₹1,099.00",₹84.96,NaN,Delivered to buyer
5,406-9292208-6725123,"Thu, 17 Jun, 2021, 9:12 pm IST",Subinita,"HOWRAH,",WEST BENGAL,SKU: HH-FOWV-5YWO,Women's Trendy Pure Leather Clutch Purse | Lea...,1,₹200.00,NaN,NaN,Delivered to buyer
6,404-5794317-7737924,"Thu, 12 Aug, 2021, 8:03 pm IST",shailendra,"ORAI,",UTTAR PRADESH,SKU: TQ-OE6K-9DIK,Ultra Slim 100% Pure Leather Men's Wallet with...,1,NaN,NaN,Cash On Delivery,Returned to seller
7,405-8702211-4054722,"Wed, 29 Sept, 2021, 2:55 pm IST",Pratima,"BAREILLY,",UTTAR PRADESH,SKU: S1-A92Q-JU3X,100% Pure Leather Shantiniketan Clutch Purse: ...,1,₹399.00,₹84.96,Cash On Delivery,Delivered to buyer
8,171-1434812-8061163,"Sat, 13 Nov, 2021, 7:37 pm IST",Ipshita,"BENGALURU,",KARNATAKA,SKU: 3F-4R9N-Z8NJ,Set of 2 Pure Leather Block Print Round Jewelr...,1,₹399.00,₹84.96,NaN,Delivered to buyer
9,171-7954707-4463549,"Mon, 9 Aug, 2021, 4:47 pm IST",A.Jayaprada,"Bhilai,",CHHATTISGARH,SKU: NU-CKZ5-4O49,Pure Leather Sling Bag with Multiple Pockets a...,1,"₹1,099.00",NaN,NaN,Delivered to buyer


In [ ]:
df.shape

(171, 12)

In [ ]:
places = ['ship_city', 'ship_state']
for i in places:
  df[i] = df[i].apply(lambda x:x.upper()) #apply함수 : 행 또는 열 전체에 원하는 연산을 적용

In [ ]:
df['ship_city'] = df['ship_city'].apply(lambda x: x.replace(',', ''))
df['ship_state'] = df['ship_state'].apply(lambda x: x.replace(',', ''))
df.head(10)

,order_no,order_date,buyer,ship_city,ship_state,sku,description,quantity,item_total,shipping_fee,cod,order_status
0,405-9763961-5211537,"Sun, 18 Jul, 2021, 10:38 pm IST",Mr.,CHANDIGARH,CHANDIGARH,SKU: 2X-3C0F-KNJE,100% Leather Elephant Shaped Piggy Coin Bank |...,1,₹449.00,NaN,NaN,Delivered to buyer
1,404-3964908-7850720,"Tue, 19 Oct, 2021, 6:05 pm IST",Minam,PASIGHAT,ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,₹449.00,₹60.18,NaN,Delivered to buyer
2,171-8103182-4289117,"Sun, 28 Nov, 2021, 10:20 pm IST",yatipertin,PASIGHAT,ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,₹449.00,₹60.18,NaN,Delivered to buyer
3,405-3171677-9557154,"Wed, 28 Jul, 2021, 4:06 am IST",aciya,DEVARAKONDA,TELANGANA,SKU: AH-J3AO-R7DN,Pure 100% Leather Block Print Rectangular Jewe...,1,NaN,NaN,Cash On Delivery,Delivered to buyer
4,402-8910771-1215552,"Tue, 28 Sept, 2021, 2:50 pm IST",Susmita,MUMBAI,MAHARASHTRA,SKU: KL-7WAA-Z82I,Pure Leather Sling Bag with Multiple Pockets a...,1,"₹1,099.00",₹84.96,NaN,Delivered to buyer
5,406-9292208-6725123,"Thu, 17 Jun, 2021, 9:12 pm IST",Subinita,HOWRAH,WEST BENGAL,SKU: HH-FOWV-5YWO,Women's Trendy Pure Leather Clutch Purse | Lea...,1,₹200.00,NaN,NaN,Delivered to buyer
6,404-5794317-7737924,"Thu, 12 Aug, 2021, 8:03 pm IST",shailendra,ORAI,UTTAR PRADESH,SKU: TQ-OE6K-9DIK,Ultra Slim 100% Pure Leather Men's Wallet with...,1,NaN,NaN,Cash On Delivery,Returned to seller
7,405-8702211-4054722,"Wed, 29 Sept, 2021, 2:55 pm IST",Pratima,BAREILLY,UTTAR PRADESH,SKU: S1-A92Q-JU3X,100% Pure Leather Shantiniketan Clutch Purse: ...,1,₹399.00,₹84.96,Cash On Delivery,Delivered to buyer
8,171-1434812-8061163,"Sat, 13 Nov, 2021, 7:37 pm IST",Ipshita,BENGALURU,KARNATAKA,SKU: 3F-4R9N-Z8NJ,Set of 2 Pure Leather Block Print Round Jewelr...,1,₹399.00,₹84.96,NaN,Delivered to buyer
9,171-7954707-4463549,"Mon, 9 Aug, 2021, 4:47 pm IST",A.Jayaprada,BHILAI,CHHATTISGARH,SKU: NU-CKZ5-4O49,Pure Leather Sling Bag with Multiple Pockets a...,1,"₹1,099.00",NaN,NaN,Delivered to buyer


In [ ]:
#변수별 결측치 개수 확인
df.isnull().sum()

order_no          0
order_date        0
buyer             0
ship_city         0
ship_state        0
sku               0
description       0
quantity          0
item_total       18
shipping_fee     26
cod             124
order_status      0
dtype: int64

In [ ]:
#결측치 핸들링 
df['shipping_fee'].fillna(df['shipping_fee'].mode()[0], inplace=True) #mode 함수로 변수에 해당하는 데이터 중 최빈값에 해당하는 값을 결측값에 대체 
df['item_total'].fillna(df['item_total'].mode()[0], inplace=True)
df['cod'].fillna('online', inplace=True) #cod에 결측값이 있다는 것은 현금, 매장 결제가 아닌 인터넷 주문을 의미 -> 따라서 결측값을 online으로 대체

In [ ]:
df.isnull().sum()

order_no        0
order_date      0
buyer           0
ship_city       0
ship_state      0
sku             0
description     0
quantity        0
item_total      0
shipping_fee    0
cod             0
order_status    0
dtype: int64

In [ ]:
df.head(10)

,order_no,order_date,buyer,ship_city,ship_state,sku,description,quantity,item_total,shipping_fee,cod,order_status
0,405-9763961-5211537,"Sun, 18 Jul, 2021, 10:38 pm IST",Mr.,CHANDIGARH,CHANDIGARH,SKU: 2X-3C0F-KNJE,100% Leather Elephant Shaped Piggy Coin Bank |...,1,₹449.00,₹84.96,online,Delivered to buyer
1,404-3964908-7850720,"Tue, 19 Oct, 2021, 6:05 pm IST",Minam,PASIGHAT,ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,₹449.00,₹60.18,online,Delivered to buyer
2,171-8103182-4289117,"Sun, 28 Nov, 2021, 10:20 pm IST",yatipertin,PASIGHAT,ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,₹449.00,₹60.18,online,Delivered to buyer
3,405-3171677-9557154,"Wed, 28 Jul, 2021, 4:06 am IST",aciya,DEVARAKONDA,TELANGANA,SKU: AH-J3AO-R7DN,Pure 100% Leather Block Print Rectangular Jewe...,1,₹449.00,₹84.96,Cash On Delivery,Delivered to buyer
4,402-8910771-1215552,"Tue, 28 Sept, 2021, 2:50 pm IST",Susmita,MUMBAI,MAHARASHTRA,SKU: KL-7WAA-Z82I,Pure Leather Sling Bag with Multiple Pockets a...,1,"₹1,099.00",₹84.96,online,Delivered to buyer
5,406-9292208-6725123,"Thu, 17 Jun, 2021, 9:12 pm IST",Subinita,HOWRAH,WEST BENGAL,SKU: HH-FOWV-5YWO,Women's Trendy Pure Leather Clutch Purse | Lea...,1,₹200.00,₹84.96,online,Delivered to buyer
6,404-5794317-7737924,"Thu, 12 Aug, 2021, 8:03 pm IST",shailendra,ORAI,UTTAR PRADESH,SKU: TQ-OE6K-9DIK,Ultra Slim 100% Pure Leather Men's Wallet with...,1,₹449.00,₹84.96,Cash On Delivery,Returned to seller
7,405-8702211-4054722,"Wed, 29 Sept, 2021, 2:55 pm IST",Pratima,BAREILLY,UTTAR PRADESH,SKU: S1-A92Q-JU3X,100% Pure Leather Shantiniketan Clutch Purse: ...,1,₹399.00,₹84.96,Cash On Delivery,Delivered to buyer
8,171-1434812-8061163,"Sat, 13 Nov, 2021, 7:37 pm IST",Ipshita,BENGALURU,KARNATAKA,SKU: 3F-4R9N-Z8NJ,Set of 2 Pure Leather Block Print Round Jewelr...,1,₹399.00,₹84.96,online,Delivered to buyer
9,171-7954707-4463549,"Mon, 9 Aug, 2021, 4:47 pm IST",A.Jayaprada,BHILAI,CHHATTISGARH,SKU: NU-CKZ5-4O49,Pure Leather Sling Bag with Multiple Pockets a...,1,"₹1,099.00",₹84.96,online,Delivered to buyer


In [ ]:
#item_total, shipping_fee 루피 기호 제거

amount = ['item_total', 'shipping_fee']
for i in amount :
  df[i] = df[i].apply(lambda x : x[1:]) #첫번째에 위치한 기호를 슬라이싱으로 제거

In [ ]:
df.head(10)

,order_no,order_date,buyer,ship_city,ship_state,sku,description,quantity,item_total,shipping_fee,cod,order_status
0,405-9763961-5211537,"Sun, 18 Jul, 2021, 10:38 pm IST",Mr.,CHANDIGARH,CHANDIGARH,SKU: 2X-3C0F-KNJE,100% Leather Elephant Shaped Piggy Coin Bank |...,1,449.00,84.96,online,Delivered to buyer
1,404-3964908-7850720,"Tue, 19 Oct, 2021, 6:05 pm IST",Minam,PASIGHAT,ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,449.00,60.18,online,Delivered to buyer
2,171-8103182-4289117,"Sun, 28 Nov, 2021, 10:20 pm IST",yatipertin,PASIGHAT,ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,449.00,60.18,online,Delivered to buyer
3,405-3171677-9557154,"Wed, 28 Jul, 2021, 4:06 am IST",aciya,DEVARAKONDA,TELANGANA,SKU: AH-J3AO-R7DN,Pure 100% Leather Block Print Rectangular Jewe...,1,449.00,84.96,Cash On Delivery,Delivered to buyer
4,402-8910771-1215552,"Tue, 28 Sept, 2021, 2:50 pm IST",Susmita,MUMBAI,MAHARASHTRA,SKU: KL-7WAA-Z82I,Pure Leather Sling Bag with Multiple Pockets a...,1,"1,099.00",84.96,online,Delivered to buyer
5,406-9292208-6725123,"Thu, 17 Jun, 2021, 9:12 pm IST",Subinita,HOWRAH,WEST BENGAL,SKU: HH-FOWV-5YWO,Women's Trendy Pure Leather Clutch Purse | Lea...,1,200.00,84.96,online,Delivered to buyer
6,404-5794317-7737924,"Thu, 12 Aug, 2021, 8:03 pm IST",shailendra,ORAI,UTTAR PRADESH,SKU: TQ-OE6K-9DIK,Ultra Slim 100% Pure Leather Men's Wallet with...,1,449.00,84.96,Cash On Delivery,Returned to seller
7,405-8702211-4054722,"Wed, 29 Sept, 2021, 2:55 pm IST",Pratima,BAREILLY,UTTAR PRADESH,SKU: S1-A92Q-JU3X,100% Pure Leather Shantiniketan Clutch Purse: ...,1,399.00,84.96,Cash On Delivery,Delivered to buyer
8,171-1434812-8061163,"Sat, 13 Nov, 2021, 7:37 pm IST",Ipshita,BENGALURU,KARNATAKA,SKU: 3F-4R9N-Z8NJ,Set of 2 Pure Leather Block Print Round Jewelr...,1,399.00,84.96,online,Delivered to buyer
9,171-7954707-4463549,"Mon, 9 Aug, 2021, 4:47 pm IST",A.Jayaprada,BHILAI,CHHATTISGARH,SKU: NU-CKZ5-4O49,Pure Leather Sling Bag with Multiple Pockets a...,1,"1,099.00",84.96,online,Delivered to buyer


In [ ]:
# 비지니스 인사이트 도출 
1. 결제방식이 구매 성공 여부에 영향을 미칠 것인가?
2. 판매 아이템 트렌드
3. 어떤 주에 판매량이 가장 높은지?
4. 어느 state에서 가장 높은 구매량을 보이는지?
5. 판매량이 가장 높은 물건
6. 반품 비율은?
7. 고객 사이에 가장 인기있는 키워드는?

SyntaxError: ignored

In [ ]:
#item_total과 shipping_fee, quantity 데이터 타입 변경
df['item_total'] = df['item_total'].apply(lambda x: x.replace(',','')) #1,099.99 에서 문자열 ,이 들어가있어서 오류 발생 = 제거 작업
i = 'int64'
f = 'float64'
df = df.astype({'item_total' : f, 'shipping_fee' : f, 'quantity' : i})

In [ ]:
# 결제방식이 구매성공 여부에 미치는 영향을 구하기 위해서는 cod, order_status, item_total

df[['cod', 'order_status', 'item_total']].groupby(['cod', 'order_status']).sum() 
#해당 결과는 cod가 인덱스로 들어가있음 -> 원하는 결과 아님

item_total
cod              order_status                  
Cash On Delivery Delivered to buyer     23812.0
                 Returned to seller      2245.0
online           Delivered to buyer     68508.0
                 Returned to seller      2694.0

In [ ]:
df[['cod', 'order_status', 'item_total']].groupby(['cod', 'order_status']).sum().reset_index() #reset_index 함수로 cod가 인덱스로 가있는 것을 제거해줌

,cod,order_status,item_total
0,Cash On Delivery,Delivered to buyer,23812.0
1,Cash On Delivery,Returned to seller,2245.0
2,online,Delivered to buyer,68508.0
3,online,Returned to seller,2694.0


In [ ]:
#시각화 라이브러리 plotply에서 히스토그램 그리기 
fig = px.histogram(df, x='cod', y='item_total', color='order_status', barmode='group', height=450) #구매성공 여부에 따른 색을 부여, 각 범주별 구매/환불을 그룹화하여 그림
fig.show()

In [ ]:
##2주차

#날짜 format
df['year'] = pd.DatetimeIndex(df['order_date']).year #년도만 추출
df['date'] = pd.DatetimeIndex(df['order_date']).date #날짜 추출
df['time'] = pd.DatetimeIndex(df['order_date']).time #시간 추출
df['month_name'] = pd.DatetimeIndex(df['date']).month_name() #월 이름 변수
df['day_name'] = pd.DatetimeIndex(df['date']).day_name() #요일 변수

In [ ]:
df.head(5)

,order_no,order_date,buyer,ship_city,ship_state,sku,description,quantity,item_total,shipping_fee,cod,order_status,year,date,time,month_name,day_name
0,405-9763961-5211537,"Sun, 18 Jul, 2021, 10:38 pm IST",Mr.,CHANDIGARH,CHANDIGARH,SKU: 2X-3C0F-KNJE,100% Leather Elephant Shaped Piggy Coin Bank |...,1,449.0,84.96,online,Delivered to buyer,2021,2021-07-18,22:38:00,July,Sunday
1,404-3964908-7850720,"Tue, 19 Oct, 2021, 6:05 pm IST",Minam,PASIGHAT,ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,449.0,60.18,online,Delivered to buyer,2021,2021-10-19,18:05:00,October,Tuesday
2,171-8103182-4289117,"Sun, 28 Nov, 2021, 10:20 pm IST",yatipertin,PASIGHAT,ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,449.0,60.18,online,Delivered to buyer,2021,2021-11-28,22:20:00,November,Sunday
3,405-3171677-9557154,"Wed, 28 Jul, 2021, 4:06 am IST",aciya,DEVARAKONDA,TELANGANA,SKU: AH-J3AO-R7DN,Pure 100% Leather Block Print Rectangular Jewe...,1,449.0,84.96,Cash On Delivery,Delivered to buyer,2021,2021-07-28,04:06:00,July,Wednesday
4,402-8910771-1215552,"Tue, 28 Sept, 2021, 2:50 pm IST",Susmita,MUMBAI,MAHARASHTRA,SKU: KL-7WAA-Z82I,Pure Leather Sling Bag with Multiple Pockets a...,1,1099.0,84.96,online,Delivered to buyer,2021,2021-09-28,14:50:00,September,Tuesday


In [ ]:
drop(df, 'order_date')
df.head(2)

,order_no,buyer,ship_city,ship_state,sku,description,quantity,item_total,shipping_fee,cod,order_status,year,date,time,month_name,day_name
0,405-9763961-5211537,Mr.,CHANDIGARH,CHANDIGARH,SKU: 2X-3C0F-KNJE,100% Leather Elephant Shaped Piggy Coin Bank |...,1,449.0,84.96,online,Delivered to buyer,2021,2021-07-18,22:38:00,July,Sunday
1,404-3964908-7850720,Minam,PASIGHAT,ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,449.0,60.18,online,Delivered to buyer,2021,2021-10-19,18:05:00,October,Tuesday


In [ ]:
#sku변수에 중복으로 적힌 sku: 문구 삭제
df['sku'] = df['sku'].apply(lambda x: x[4:])
df.head(3)

,order_no,buyer,ship_city,ship_state,sku,description,quantity,item_total,shipping_fee,cod,order_status,year,date,time,month_name,day_name
0,405-9763961-5211537,Mr.,CHANDIGARH,CHANDIGARH,2X-3C0F-KNJE,100% Leather Elephant Shaped Piggy Coin Bank |...,1,449.0,84.96,online,Delivered to buyer,2021,2021-07-18,22:38:00,July,Sunday
1,404-3964908-7850720,Minam,PASIGHAT,ARUNACHAL PRADESH,DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,449.0,60.18,online,Delivered to buyer,2021,2021-10-19,18:05:00,October,Tuesday
2,171-8103182-4289117,yatipertin,PASIGHAT,ARUNACHAL PRADESH,DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,449.0,60.18,online,Delivered to buyer,2021,2021-11-28,22:20:00,November,Sunday


In [ ]:
# date에 따른 총 판매가격

df_sales = df[['date', 'item_total']].groupby('date').sum().reset_index() #date를 기준으로 item값 합치고, 인덱스를 새롭게 리셋해 0,1,2의 인덱스 값을 가짐
df_sales.head(3)

,date,item_total
0,2021-06-13,175.0
1,2021-06-16,525.0
2,2021-06-17,200.0


In [ ]:
#date별 item_total값 시각화

fig = px.line(df_sales, x='date', y='item_total', title='Sales ocer the period')
fig.show()

#12월 21일 아이템 판매량이 가장 높고, 대체적으로 12월달의 판매액이 큰 것을 알 수 있음 

In [ ]:
df_month = df[['month_name', 'item_total', 'day_name', 'year']].groupby(['year','month_name','day_name']).sum().reset_index()
df_month.head(5)

,year,month_name,day_name,item_total
0,2021,August,Friday,2647.0
1,2021,August,Monday,2448.0
2,2021,August,Sunday,649.0
3,2021,August,Thursday,449.0
4,2021,August,Tuesday,399.0


In [ ]:
#위의 내용 가지고 월별 요일 판매량을 막대그래프로 확인하고자 함
#월별, 요일별은 범주형 = 막대그래프

fig_box = px.bar(df_month, x='month_name', y='item_total', color='day_name', title='Month-wise Sales')
fig_box.show()

In [ ]:
#선버스트 차트 : 계층 구조를 나타내기에 적합한 그래프 하나의 원이 하나의 계층을 나타내며, 안쪽에 있는 원일 수록 가장 높은 수준의 계층을 나타냄

#데이터 전처리 
df_sunburst = df[['ship_city', 'ship_state', 'item_total']].groupby(['ship_city', 'ship_state']).sum().reset_index()
df_sunburst.head(3)
## 선버스트 차트는 데이터를 내림차순으로 정렬하여 진행하는 것이 바람직
df_sunburst.sort_values(by='item_total', ascending=False, inplace=True) #inplace=True는 변경된 내용을 기존의 데이터 프레임에 바로 덮어씌우겠다는 의미

In [ ]:
fig = px.sunburst(data_frame = df_sunburst, path=['ship_state', 'ship_city'], values='item_total', title='Region-wise orders')
fig.show()

In [ ]:
#주문 수가 높은 상위 5개 지역
top_city = df.groupby('ship_city').size().reset_index().rename(columns={0:'Total'}).sort_values('Total', ascending=False).head(5) #두번째 변수인 0은 각 지역별 주문횟수를 나타내므로 rename을 통해 total로 재변환

In [ ]:
#파이차트를 통한 지역별 주문량 상위 5개 출력
fig = px.pie(top_city, values='Total', names='ship_city', color_discrete_sequence=px.colors.sequential.RdBu, title='Top 5 ordering cities')
fig.show()

In [ ]:
#상위 5개 products 파이차트로 그리기
#범주 별 개수를 세알릴때는 sum이 아닌 size()함수 사용
top_products = df.groupby('sku').size().reset_index().rename(columns={0:'Total'}).sort_values('Total', ascending=False).head(5)
top_products.head()

,sku,Total
22,DN-0WDX-VYOT,33
0,0M-RFE6-443C,10
42,SB-WDQN-SDN9,8
19,CR-6E69-UXFW,8
2,2X-3C0F-KNJE,7


In [ ]:
fig = px.pie(top_products, values='Total', names='sku', color_discrete_sequence=px.colors.sequential.BuGn_r, title='Top 5 Products')
fig.show()